In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.quantization import quantize_dynamic

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
pip install torchvision

In [ ]:
# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [4]:
# Load and preprocess MNIST data
transform     = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = MNIST(root='./data', train=False, download=True, transform=transform)


100%|██████████████████████████████████████████████████████████████████| 9912422/9912422 [00:00<00:00, 11577192.09it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 28855572.61it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 3177067.51it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<?, ?it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [5]:
# Create DataLoader for training and test datasets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [6]:
# Initialize the model, criterion, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.view(-1, 28 * 28))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [8]:
# Evaluate the unquantized model on the test data
model.eval()
correct_unquantized = 0
total_unquantized = 0

In [9]:
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs_unquantized = model(inputs.view(-1, 28 * 28))
        _, predicted_unquantized = torch.max(outputs_unquantized, 1)
        total_unquantized += labels.size(0)
        correct_unquantized += (predicted_unquantized == labels).sum().item()

accuracy_unquantized = correct_unquantized / total_unquantized
print(f'Accuracy of the unquantized model on the test data: {100 * accuracy_unquantized:.2f}%')


Accuracy of the unquantized model on the test data: 96.53%


In [10]:
# Quantize the model dynamically
quantized_model = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)


In [11]:
# Evaluate the quantized model on the test data
quantized_model.eval()
correct_quantized = 0
total_quantized = 0

In [12]:
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs_quantized = quantized_model(inputs.view(-1, 28 * 28))
        _, predicted_quantized = torch.max(outputs_quantized, 1)
        total_quantized += labels.size(0)
        correct_quantized += (predicted_quantized == labels).sum().item()

accuracy_quantized = correct_quantized / total_quantized
print(f'Accuracy of the quantized model on the test data: {100 * accuracy_quantized:.2f}%')

Accuracy of the quantized model on the test data: 96.57%
